# 2A.ml - Clustering

Ce notebook utilise les données des vélos de Chicago [Divvy Data](https://www.divvybikes.com/system-data). Il s'inspire du challenge créée pour découvrir les habitudes des habitantes de la ville [City Bike](http://www.xavierdupre.fr/app/ensae_projects/helpsphinx/challenges/city_bike.html). L'idée est d'explorer plusieurs algorithmes de clustering et de voire comment trafiquer les données pour les faire marcher et en tirer quelques apprentissages.

In [1]:
from jyquickhelper import add_notebook_menu
add_notebook_menu()

In [3]:
%matplotlib inline

## Les données

Elles ont été prétraitées selon le notebook [Bike Pattern 2](http://www.xavierdupre.fr/app/ensae_projects/helpsphinx/notebooks/city_bike_solution_cluster_start.html). Elles représentent la distribution du nombre de vélos partant (*startdist*) et arrivant (*stopdist*). On utilise le clustering pour découvrir les différents usages des habitants de Chicago avec pour intuition le fait que les habitants de Chicago utilise majoritairement les vélos pour aller et venir entre leur appartement et leur lieu de travail. Cette même idée mais à Paris est illustrée par ce billet de blog : [Busy areas in Paris](http://www.xavierdupre.fr/blog/2013-09-26_nojs.html).

In [4]:
from pyensae.datasource import download_data
file = download_data("features_bike_chicago.zip")
file

['features_bike_chicago.txt']

In [6]:
import pandas
features = pandas.read_csv("features_bike_chicago.txt", sep="\t", encoding="utf-8", low_memory=False, header=[0,1])
features.columns = ["station_id", "station_name", "weekday"] + list(features.columns[3:])
features.head()

,station_id,station_name,weekday,"(startdist, 00:00:00)","(startdist, 00:10:00)","(startdist, 00:20:00)","(startdist, 00:30:00)","(startdist, 00:40:00)","(startdist, 00:50:00)","(startdist, 01:00:00)",...,"(stopdist, 22:20:00)","(stopdist, 22:30:00)","(stopdist, 22:40:00)","(stopdist, 22:50:00)","(stopdist, 23:00:00)","(stopdist, 23:10:00)","(stopdist, 23:20:00)","(stopdist, 23:30:00)","(stopdist, 23:40:00)","(stopdist, 23:50:00)"
0,2.0,Michigan Ave & Balbo Ave,0.0,0.003756,0.000939,0.002817,0.000000,0.000000,0.003756,0.000000,...,0.004381,0.002191,0.004381,0.002191,0.004381,0.004381,0.005476,0.002191,0.000000,0.005476
1,2.0,Michigan Ave & Balbo Ave,1.0,0.000000,0.000000,0.001106,0.001106,0.001106,0.002212,0.000000,...,0.009371,0.012048,0.006693,0.004016,0.005355,0.006693,0.002677,0.000000,0.000000,0.000000
2,2.0,Michigan Ave & Balbo Ave,2.0,0.001357,0.002714,0.000000,0.001357,0.000000,0.005427,0.000000,...,0.002907,0.002907,0.015988,0.005814,0.001453,0.001453,0.011628,0.000000,0.000000,0.007267
3,2.0,Michigan Ave & Balbo Ave,3.0,0.000000,0.004144,0.000000,0.000000,0.002762,0.004144,0.000000,...,0.009274,0.003091,0.003091,0.007728,0.001546,0.003091,0.009274,0.001546,0.007728,0.001546
4,2.0,Michigan Ave & Balbo Ave,4.0,0.000000,0.000000,0.000000,0.002846,0.000000,0.000000,0.000949,...,0.008214,0.001027,0.006160,0.004107,0.015400,0.006160,0.002053,0.006160,0.007187,0.000000


## Exercice 1 : petits clusters

Quelques éléments de réponse.

Les petits clusters sont des stations peu utilisées. Le problème quand on calcule une distribution et qu'on la normalise, typiquement celles des arrivées de vélos à une stations, c'est qu'on perd l'information du nombre de vélos ayant servi à l'estimer. 

L'algorithme [k-means++](https://en.wikipedia.org/wiki/K-means%2B%2B) commence à choisir les points les plus éloignés des unss des autres et cela tombe sur ces stations peu utilisées. Vu le nombre de dimension, 288, les distances sont rapidement très grandes.

Une idée consiste à retirer ces stations peu utilisées ou boucler sur un algorithme du type :

1. Faire tourner les k-means.
2. Retirer les stations dans des clusters isolées.
3. Retour à l'étape 1 jusqu'à ce que cela soit interprétable.

## Exercice 2 : autres types de clustering

Quelques éléments de réponse.

L'algorithme [DBScan](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html) ne fonctionne pas sur ces données. Une des raisons que la distance dans un espace vectoriels avec tant de dimensions n'est pas loin d'une information type binaire : différent partout ou identique presque partout. Qu'un vélo arrive 9h10 ou à 9h20, cela n'a pas beaucoup d'importance pour notre problème, pourtant, la distance choisi lui donnera autant d'importance que si le vélo était arrivé à 10h du soir. Pour éviter cela, il faut lisser les distributions avec une moyenne mobile.